# Visualizing Embedding Evolution During Training

In this notebook, we train an MLP on MNIST and visualize how hidden representations (embeddings) evolve over training using t-SNE. This gives insight into how class structure emerges in the latent space over time.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import random

# Reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [ ]:
transform = transforms.ToTensor()

train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

#train_subset, _ = torch.utils.data.random_split(train_data, [10000, len(train_data)-10000])
#train_loader = DataLoader(train_subset, batch_size=128, shuffle=True)

train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False)

In [ ]:
from torch.utils.data import Subset

# First, get all test labels
test_targets = test_data.targets.numpy()

# Define how many samples per class
samples_per_class = 200
num_classes = 10

# Collect indices (balanced subset)
selected_indices = []
for class_id in range(num_classes):
    class_indices = np.where(test_targets == class_id)[0]
    chosen = np.random.choice(class_indices, size=samples_per_class, replace=False)
    selected_indices.extend(chosen)

# Create a test subset DataLoader (shuffling not needed)
test_subset = Subset(test_data, selected_indices)
test_subset_loader = DataLoader(test_subset, batch_size=128, shuffle=False)

In [ ]:
print(len(train_data))
print(len(test_data))
print(len(test_subset))

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim=28*28, hidden_dim=128, num_classes=10):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x, return_embedding=False):
        x = x.view(x.size(0), -1)
        h = F.relu(self.fc1(x))
        out = self.fc2(h)
        if return_embedding:
            return out, h
        return out

model = MLP().to(device)

In [ ]:
# Training parameters
epochs = 20
learning_rate = 0.001

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# Metrics
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

# Embeddings
# train_embeddings_all = []
# test_embeddings_all = []
# train_labels_all = []
# test_labels_all = []
test_subset_embeddings = []
test_subset_labels = []


for epoch in range(epochs):
    # === Train phase ===
    model.train()
    epoch_train_loss = 0
    correct_train = 0
    total_train = 0
    epoch_train_embeddings = []
    epoch_train_labels = []

    for data, target in train_loader:
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output, embedding = model(data, return_embedding=True)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        # Metrics
        epoch_train_loss += loss.item()
        _, preds = torch.max(output, dim=1)
        correct_train += (preds == target).sum().item()
        total_train += target.size(0)

        with torch.no_grad():
            batch_embeddings = []
            batch_labels = []

            for data_sub, target_sub in test_subset_loader:
                data_sub, target_sub = data_sub.to(device), target_sub.to(device)
                _, emb = model(data_sub, return_embedding=True)
                batch_embeddings.append(emb.cpu().numpy())
                batch_labels.append(target_sub.cpu().numpy())

            test_subset_embeddings.append(np.concatenate(batch_embeddings, axis=0))
            test_subset_labels.append(np.concatenate(batch_labels, axis=0))

        # Store embeddings
        #epoch_train_embeddings.append(embedding.cpu().detach().numpy())
        #epoch_train_labels.append(target.cpu().numpy())

    train_loss = epoch_train_loss / len(train_loader)
    train_acc = correct_train / total_train
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    #train_embeddings_all.append(np.concatenate(epoch_train_embeddings, axis=0))
    #train_labels_all.append(np.concatenate(epoch_train_labels, axis=0))

    # === Eval phase ===
    model.eval()
    epoch_val_loss = 0
    correct_val = 0
    total_val = 0
    epoch_val_embeddings = []
    epoch_val_labels = []

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output, _ = model(data, return_embedding=True)
            loss = criterion(output, target)

            epoch_val_loss += loss.item()
            _, preds = torch.max(output, dim=1)
            correct_val += (preds == target).sum().item()
            total_val += target.size(0)

            #epoch_val_embeddings.append(embedding.cpu().numpy())
            #epoch_val_labels.append(target.cpu().numpy())

    val_loss = epoch_val_loss / len(test_loader)
    val_acc = correct_val / total_val
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)
    #test_embeddings_all.append(np.concatenate(epoch_val_embeddings, axis=0))
    #test_labels_all.append(np.concatenate(epoch_val_labels, axis=0))

    # === Print summary ===
    print(f"Epoch [{epoch+1}/{epochs}] "
          f"| Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} "
          f"| Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

# Epoch [50/50] | Train Loss: 0.0562, Acc: 0.9851 | Val Loss: 0.0848, Acc: 0.9739
# Epoch [10/10] | Train Loss: 0.0370, Acc: 0.9894 | Val Loss: 0.0796, Acc: 0.9764

In [ ]:
# Plotting Training and Validation Loss
plt.plot(range(epochs), train_losses, label='Train Loss', color='blue')
plt.plot(range(epochs), val_losses, label='Validation Loss', color='red')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss over Epochs')
plt.legend()
plt.show()

# tSNE Computation

In [ ]:
# Labels should be the same everywhere
print(labels[0])
print(labels[1])

In [ ]:
embeddings[0]

In [ ]:
# Final epoch's data
final_embeddings = embeddings[-1]
final_labels = labels[-1]

# Convert to NumPy arrays (if not already)
final_embeddings = np.array(final_embeddings)
final_labels = np.array(final_labels)

# How many samples per class
samples_per_class = 200
num_classes = 10

# Collect indices
selected_indices = []
for class_id in range(num_classes):
    class_indices = np.where(final_labels == class_id)[0]
    chosen = np.random.choice(class_indices, size=samples_per_class, replace=False)
    selected_indices.extend(chosen)

# Subsample embeddings and labels
sampled_embeddings = final_embeddings[selected_indices]
sampled_labels = final_labels[selected_indices]

In [ ]:
# Optional: Refresh tSNE
epoch = 10

# Final epoch's data
final_embeddings = embeddings[epoch]
final_labels = labels[epoch]

# Subsample embeddings and labels
sampled_embeddings = final_embeddings[selected_indices]
sampled_labels = final_labels[selected_indices]

In [ ]:
# Apply t-SNE
tsne = TSNE(n_components=2, random_state=42)
tsne_results = tsne.fit_transform(sampled_embeddings)

# Plot
plt.figure(figsize=(8, 8))
plt.scatter(tsne_results[:, 0], tsne_results[:, 1], c=sampled_labels, cmap='tab10', alpha=0.8)
plt.title('t-SNE of Final Epoch Embeddings (Balanced Sample)')
plt.xlabel('t-SNE Dimension 1')
plt.ylabel('t-SNE Dimension 2')
plt.colorbar(label='Digit Label')
plt.show()

# Animation

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from sklearn.manifold import TSNE

# === Compute t-SNE per epoch using selected_indices ===
tsne_results = []

# Epoch 0: PCA init
tsne = TSNE(n_components=2, init='pca', random_state=42)
result = tsne.fit_transform(embeddings[0][selected_indices])
tsne_results.append(result)

# Epochs 1+: use previous output for init
for epoch in range(1, len(embeddings)):
    tsne = TSNE(n_components=2, init=tsne_results[-1], random_state=42)
    result = tsne.fit_transform(embeddings[epoch][selected_indices])
    tsne_results.append(result)

# === Animation Setup ===
fig, ax = plt.subplots(figsize=(8, 8))
scatter = ax.scatter([], [], c=[], cmap='tab10', alpha=0.7)
ax.set_xlim(-100, 100)
ax.set_ylim(-100, 100)
ax.set_xlabel('t-SNE Dimension 1')
ax.set_ylabel('t-SNE Dimension 2')
ax.set_title('t-SNE Embedding Evolution')

# Use sampled_labels once — they're fixed
def update(frame):
    scatter.set_offsets(tsne_results[frame])
    scatter.set_array(sampled_labels)
    ax.set_title(f't-SNE Embedding – Epoch {frame + 1}')
    return scatter,

ani = animation.FuncAnimation(fig, update, frames=len(tsne_results), interval=1000, blit=True)
#plt.show()

In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# Static figure (inline mode is fine)
fig, ax = plt.subplots(figsize=(8, 8))

# Plot the initial frame (epoch 0)
sc = ax.scatter(
    tsne_results[0][:, 0], tsne_results[0][:, 1],
    c=sampled_labels, cmap='tab10', alpha=0.7
)
ax.set_xlim(-100, 100)
ax.set_ylim(-100, 100)
ax.set_xlabel('t-SNE Dimension 1')
ax.set_ylabel('t-SNE Dimension 2')
title = ax.set_title("t-SNE Embedding – Epoch 1")

# Update function for slider
def update_epoch(epoch):
    print(epoch)
    sc.set_offsets(tsne_results[epoch])
    title.set_text(f't-SNE Embedding – Epoch {epoch + 1}')
    fig.canvas.draw_idle()

# IntSlider widget
epoch_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(tsne_results) - 1,
    step=1,
    description='Epoch:',
    continuous_update=True
)

# Connect slider to update function
widgets.interact(update_epoch, epoch=epoch_slider)

In [ ]:
import os

# === Define experiment info ===
model_name = model._get_name()
# Build a base name with settings
base_name = f"tsne_{model_name}_e{epochs}_lr{learning_rate:.0e}"  # e.g., tsne_MLP_e10_lr1e-03

# Auto-increment file index
i = 1
while os.path.exists(f"{base_name}_{i:03}.gif"):
    i += 1
filename = f"{base_name}_{i:03}.gif"

# Save the animation
ani.save(filename, writer='pillow')
print(f"Saved animation as {filename}")

# Video Animation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from sklearn.manifold import TSNE
import os

# === Interpolation between t-SNE results ===
def interpolate_tsne(embeddings_list, steps=10):
    interpolated = []
    for a, b in zip(embeddings_list[:-1], embeddings_list[1:]):
        for alpha in np.linspace(0, 1, steps, endpoint=False):
            interp = (1 - alpha) * a + alpha * b
            interpolated.append(interp)
    interpolated.append(embeddings_list[-1])  # include final frame
    return interpolated

# === Compute t-SNE per epoch using selected_indices ===
tsne_results = []

# Epoch 0
tsne = TSNE(n_components=2, init='pca', random_state=42)
result = tsne.fit_transform(embeddings[0][selected_indices])
tsne_results.append(result)

# Subsequent epochs
for epoch in range(1, len(embeddings)):
    tsne = TSNE(n_components=2, init=tsne_results[-1], random_state=42)
    result = tsne.fit_transform(embeddings[epoch][selected_indices])
    tsne_results.append(result)

# === Interpolate between epochs ===
tsne_results_interp = interpolate_tsne(tsne_results, steps=5)

# === Animation setup ===
fig, ax = plt.subplots(figsize=(8, 8))
scatter = ax.scatter([], [], cmap='tab10', alpha=0.7)
ax.set_xlim(-100, 100)
ax.set_ylim(-100, 100)
ax.set_xlabel('t-SNE Dimension 1')
ax.set_ylabel('t-SNE Dimension 2')
title = ax.set_title('t-SNE Embedding Evolution')

def update(frame):
    scatter.set_offsets(tsne_results_interp[frame])
    scatter.set_array(sampled_labels)
    title.set_text(f't-SNE Embedding Evolution – Frame {frame + 1}')
    return scatter,

ani = animation.FuncAnimation(fig, update, frames=len(tsne_results_interp), interval=100, blit=True)

In [ ]:
from IPython.display import HTML
HTML(ani.to_jshtml())

In [ ]:
# === Save as MP4 (with ffmpeg) ===
model_name = model._get_name()
lr = learning_rate
base_name = f"tsne_{model_name}_e{epochs}_lr{lr:.0e}"
i = 1
while os.path.exists(f"{base_name}_{i:03}.mp4"):
    i += 1
filename = f"{base_name}_{i:03}.mp4"

ani.save(filename, writer='ffmpeg', dpi=150)
print(f"Saved animation as {filename}")